In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import *

In [0]:
df = spark.read.table("samples.accuweather.historical_hourly_metric")

In [0]:
df.groupBy("city_name").agg(sum("cloud_base_height"), avg("cloud_base_height")).display()

In [0]:
window_spec = Window.orderBy(desc("city_name"))

df.withColumn("rank", rank().over(window_spec)).select("city_name", "rank").display()

In [0]:
df.alias("df").join(df.alias("df2"), col("df.city_name") == col("df2.city_name"), "left").display()

In [0]:
transactions_df = spark.read.table("samples.bakehouse.sales_transactions")
suppliers_df = spark.read.table("samples.bakehouse.sales_suppliers")
franchise_df = spark.read.table("samples.bakehouse.sales_franchises")
customer_df = spark.read.table("samples.bakehouse.sales_customers")

In [0]:
fact_df = franchise_df.join(transactions_df, on = transactions_df.franchiseID == franchise_df.franchiseID, how = "inner")

In [0]:
full_join = franchise_df.withColumnRenamed("name", "franchise_name")\
                        .join(suppliers_df.select("supplierID", col("name").alias("supplier_name")), 
                              on = franchise_df.supplierID == suppliers_df.supplierID, 
                              how = "full_outer"
                             )
                        

non_matching = full_join.filter(col("franchiseID").isNull() | col("supplier_name").isNull())\
                        .select("franchiseID","franchise_name","supplier_name")


non_matching.display()